In [22]:
#!pip install windrose

In [23]:
import requests
import json
import folium
from folium.plugins import MousePosition
import matplotlib.pyplot as plt
import numpy as np
import datetime
from ipywidgets import interact
import ipywidgets as widgets
from windrose import WindroseAxes

In [24]:
# Map of France
frmap = folium.Map(
    location=[47,4],
    tiles='cartodbpositron',
    zoom_start=6,
    control_scale=True
)
MousePosition().add_to(frmap)

In [25]:
# Convert degrees to cardinal direction
def degree_to_cardinal(theta):
    if  11.25>=theta>=348.75:
        return 'N'
    elif 11.25<theta<= 33.75:
        return 'NNE'
    elif 33.75 <theta<= 56.25:
        return 'NE'
    elif 56.25 <theta<= 78.75:
        return 'ENE'
    elif 78.75 <theta<= 101.25:
        return 'E'
    elif 101.25 <theta<= 123.75:
        return 'ESE'
    elif 123.75 <theta<= 146.25:
        return 'SE'
    elif 146.25 <theta<= 168.75:
        return 'SSE'
    elif 168.75 <theta<= 191.25:
        return 'S'
    elif 191.25 <theta<= 213.75:
        return 'SSW'
    elif 213.75 <theta<= 236.25:
        return 'SW'
    elif 236.25 <theta<= 258.75:
        return 'WSW'
    elif 258.75 <theta<= 281.25:
        return 'W'
    elif 281.25 <theta<= 303.75:
        return 'WNW'
    elif 303.75  <theta<= 326.25:
        return 'NW'
    elif 326.25 <theta<= 348.75:
        return 'NNW'

In [26]:
# Get a 7 day hourly prediction
# Here we get temperature and relative humidity within 2m, precipitation, wind speed and wind direction within 10 m
locations=[43.6184,5.306]
url="https://api.open-meteo.com/v1/forecast?latitude="+str(locations[0])+"&longitude="+str(locations[1])+"&hourly=temperature_2m,relativehumidity_2m,precipitation,windspeed_10m,winddirection_10m"
x = requests.get(url)
y = json.loads(x.text)
x.close()

In [27]:
# check keys
y.keys()

dict_keys(['hourly', 'generationtime_ms', 'elevation', 'utc_offset_seconds', 'hourly_units', 'latitude', 'longitude'])

In [28]:
# check hourly data keys
hourly=y['hourly']
hourly.keys()

dict_keys(['precipitation', 'relativehumidity_2m', 'windspeed_10m', 'winddirection_10m', 'time', 'temperature_2m'])

In [29]:
# interactive plots
def f(x):
    plt.plot(np.arange(0,24,1),x[:24])
    plt.xlabel('Hours', fontsize=18)
interact(f,x=hourly)

interactive(children=(Dropdown(description='x', options={'precipitation': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…

<function __main__.f(x)>

In [30]:
# Display on map
# TODO: convert time and display current data not the first
text = ' '.join(['Temperature:', str(hourly['temperature_2m'][0]), '°C<br> Wind direction:',str(degree_to_cardinal(hourly['winddirection_10m'][0])),
                 '<br> Wind speed :',str(hourly['windspeed_10m'][0]),'km/h <br> Relative humidity:',
                str(hourly['relativehumidity_2m'][0]),'%<br> Elevation:',str(y['elevation']), 'm'])

iframe = folium.IFrame(text,
                       width=300,
                       height=120)

popup = folium.Popup(iframe,
                     max_width=300)

marker = folium.Marker([44.6184747,4.3061143],icon=folium.Icon(color='red', icon='fire', prefix='glyphicon'),
                       popup=popup).add_to(frmap)
frmap